In [ ]:
!pip install transformers

In [10]:
import pandas as pd

In [13]:
test_df = pd.read_csv("/content/teknofest_preprocessed_data_5fold.csv")
test_df = test_df[test_df.kfold == 4]
test_df

,id,text,is_offensive,target,kfold
7,b9122be9-5a9a-491e-8ecc-998ac05c6ef0,kadın yönetici sayısı oldukça olduğu şirket iy...,1,SEXIST,4
10,03d5a6fe-ea99-444b-975a-63b4ccb10f29,suriyeliler yozlaşıp toplumda huzuru bozuyorlar,1,RACIST,4
16,30d96f50-dee8-4fcf-86ea-f241af78bdc7,kadar terbiyesizsin,1,INSULT,4
17,96824039-d49c-4c42-84e1-56c2bb118d10,özsaygısı düşük birisin,1,INSULT,4
22,c99b56c2-f683-48ac-90d4-f540f64c9328,köpek havlayıp duruyorsun,1,INSULT,4
...,...,...,...,...,...
12440,9b0ab0f1-1ee0-4a03-8de8-83372f41d6b2,ermenilerin böyle sorunu var,0,OTHER,4
12445,b8c41fd2-8b07-4443-a51c-054059f1c559,nedenle kendi çıkarımız avrupalılar türkiyeyi ...,0,OTHER,4
12448,67dab3e4-26f9-408d-97ff-51600d638851,hiçbir tane mi ceren gözlü kürt kızı yok,0,OTHER,4
12459,513a7e6d-4207-4a16-9b47-972f26e23cfe,hayalleri gerçek etmek birisinin delilik yapma...,0,OTHER,4


In [3]:
model1 = "/content/gdrive/MyDrive/teknofest-multiclass-model/bert-base-turkish-cased-mean-nli-stsb-tr-5fold-32maxl-32tb-64vb-5ep-001wd-1e5lr-3validfold-4testfold-corrected-misslabels"
model2 = "/content/gdrive/MyDrive/teknofest-multiclass-model/bert-base-turkish-uncased-5fold-32maxl-32tb-64vb-5ep-001wd-1e5lr-3validfold-4testfold"
model3 = "/content/gdrive/MyDrive/teknofest-multiclass-model/electra-base-turkish-cased-discriminator-5fold-32maxl-32tb-64vb-5ep-001wd-1e5lr-3validfold-4testfold"

In [4]:
from transformers import pipeline

classifier1 = pipeline("text-classification", model=model1)
classifier2 = pipeline("text-classification", model=model2)
classifier3 = pipeline("text-classification", model=model3)

In [ ]:
[{'label': 'SEXIST', 'score': 0.9890580773353577}],[{'label': 'SEXIST', 'score': 0.9890580773353577}],

In [23]:
def prediction_postprocesser(prediction:list) -> str:
  return prediction[0]["label"]

In [35]:
from collections import Counter
def ensemble_prediction(text:str) -> str:
  predictions = []

  resp1, resp2, resp3 = classifier1(text),classifier2(text),classifier3(text)

  predictions.extend(resp1)
  predictions.extend(resp2)
  predictions.extend(resp3)

  predicted_labels = [d['label'] for d in predictions]

  counter_list = Counter(predicted_labels)

  final_predicted_label, count = counter_list.most_common(1)[0]

  if count == 1:
    final_predicted_label = max(predictions, key=lambda x: x['score'])['label']

  return final_predicted_label

In [60]:
def ensemble_prediction_v2(text:str) -> str:
    predictions = []

    resp1, resp2, resp3 = classifier1(text),classifier2(text),classifier3(text)

    predictions.extend(resp1)
    predictions.extend(resp2)
    predictions.extend(resp3)

    # Her bir label için bir sözlük oluştur
    label_skorları = defaultdict(lambda: {'skorlar': [], 'sayı': 0})

    # Sözlüklerin skorlarını topla ve sayısını say
    for d in predictions:
        label = d['label']
        skor = d['score']
        label_skorları[label]['skorlar'].append(skor)
        label_skorları[label]['sayı'] += 1

    # Label'ların skorlarının ortalamasını al
    ortalama_skorlar = {}
    for label, veriler in label_skorları.items():
        skorlar = veriler['skorlar']
        sayı = veriler['sayı']
        ortalama_skorlar[label] = sum(skorlar) / sayı

    # En yüksek ortalama skora sahip label'i seç
    en_yuksek_ortalama_skor_label = max(ortalama_skorlar, key=lambda x: (ortalama_skorlar[x], max([d['score'] for d in predictions if d['label'] == x])))
  
    return en_yuksek_ortalama_skor_label


In [45]:
from tqdm import tqdm

ensemble_predictions = []

for text in tqdm(test_df['text'].tolist()):
    pred = ensemble_prediction(text)
    ensemble_predictions.append(pred)

100%|██████████| 2492/2492 [13:56<00:00,  2.98it/s]


In [49]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

gold = test_df['target'].tolist()

print("Accuracy score: ", accuracy_score(gold,ensemble_predictions))
print("f1_score score: ", f1_score(gold,ensemble_predictions, average="macro"))

Accuracy score:  0.9382022471910112
f1_score score:  0.9391720761449192


In [50]:
test_df["ensemble_predictions"] = ensemble_predictions

In [53]:
test_df[(test_df.target !=test_df.ensemble_predictions)].to_excel("ensemble_misslabels_v1.xlsx", index=False)

In [62]:
from tqdm import tqdm

ensemble_predictions_v2 = []

for text in tqdm(test_df['text'].tolist()):
    pred = ensemble_prediction_v2(text)
    ensemble_predictions_v2.append(pred)

100%|██████████| 2492/2492 [14:10<00:00,  2.93it/s]


In [63]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

gold = test_df['target'].tolist()

print("Accuracy score: ", accuracy_score(gold,ensemble_predictions_v2))
print("f1_score score: ", f1_score(gold,ensemble_predictions_v2, average="macro"))

Accuracy score:  0.9382022471910112
f1_score score:  0.9387918059089746


In [64]:
test_df["ensemble_predictions_v2"] = ensemble_predictions_v2

In [65]:
test_df[(test_df.target !=test_df.ensemble_predictions_v2)].to_excel("ensemble_misslabels_v2.xlsx", index=False)

In [68]:
test_df[(test_df.ensemble_predictions !=test_df.ensemble_predictions_v2)].to_excel("ensemble_misslabels_conflicts_between_ensembles.xlsx", index=False)

In [ ]:
test_df[(test_df.target ==test_df.ensemble_predictions) & (test_df.ensemble_predictions !=test_df.ensemble_predictions_v2)]

In [77]:
text = "amk kürdü"
resp1, resp2, resp3 = classifier1(text),classifier2(text),classifier3(text)
print("*"*100)
print("text: ", text)
print("resp1: ", resp1)
print("resp2: ", resp2)
print("resp3: ", resp3)
print("*"*100)

****************************************************************************************************
text:  amk kürdü
resp1:  [{'label': 'RACIST', 'score': 0.7063655257225037}]
resp2:  [{'label': 'PROFANITY', 'score': 0.9928267002105713}]
resp3:  [{'label': 'RACIST', 'score': 0.8985764980316162}]
****************************************************************************************************


In [ ]:
for text, target in zip(test_df["text"].tolist(),test_df["target"].tolist()):

  resp1, resp2, resp3 = classifier1(text),classifier2(text),classifier3(text)
  print("*"*100)
  print("text: ", text)
  print("gold label: ", target)
  print("resp1: ", resp1)
  print("resp2: ", resp2)
  print("resp3: ", resp3)
  print("*"*100)

  print("\n")

  
